In [1]:
# keras module for building LSTM 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability


import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-05-26 09:26:43.313526: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 09:26:43.360129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 09:26:44.073412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
corpus=[
    '평소에 운동을 열심히 했다는 점과 또 다른 나라에 뒤처지지 않는 연습량과 또한 서로 간의 화합과 위로도 잘 이루어졌을 거라고 생각해.',
  '그리고 그 상황에서도 당황하지 않고 침착하게 이루어나가야 된다는 생각을 평소에 하고 있었기 때문에 원활하게 이루어지고 좋은 결과를 얻 을 수 있지 않았을까라는 생각이 들어.',
  '그래서 나는 그 부분에 대해서 이전에 쇼트트랙 경기에서 협동이 제대로 이뤄지지 않아서 논란이 생긴 부분이 있었단 말이야.',
  '그 부분에서 실제 경기 중에 뒤처져서 안타까운 선수가 있었어.',
  '넌 그 뉴스 기사를 본 적이 있어?',
    '나는 피겨스케이트와 쇼트트랙은 유사한 종목이지만 개인이 한다는 점과 네 명이서 협동을 해야 된다는 부분에 큰 차이점이 있다고 생각하거든.',
  '그래서 그 부분에서 서로 협동이 잘 이루어지야 되는데 그 경우에는 아마 내 생각에는 선수 간의 화합이 평소에도 잘 이루어지지 않았던 거 같아.',
  '왜냐하면은 만약에 화합이 이루어졌다면은 넘어졌더라도 다시 다시 침착하게 속도를 맞추어 나가면 되는 일인데 그~ 당시에는 먼저 큰 경기이기도 하고 또 나라 대표로 나갔기 때문에 사실 그 넘어졌다는 게 중요하지 않고 넘어지지 않은 사람들이라도 잘해야 된다는 그런 압박감에 이루어짔던 안타까운 경우가 아닐까 라는 생각이 들어.',
  '그래서 나의 경우에는 지금은 은퇴해서 활동하지 않지만 피겨스케이터로 유명한 김연아 선수가 있잖아.',
  '김연아 선수에게 초점이 맞춰져서 쇼트트랙이 제대로 응원받지 않는다고 생각을 하는데 지금은 그래도 김연아 선수가 은퇴해서 쇼트트랙이 조금은 초점이 맞춰지겠지만 너는 그 쇼트트랙에 대해서 어떻게 인식되고 있을까 라고 궁금해.',
    '나는 그 근성에 대해서 일단 피겨스케이트로 일단 한 번 스포트라이트를 받았고 또 김연아 선수가 우리나라를 더 유명하게 해줬기 때문에 그 이후의 선수들에게도 사실 부담감이 있을 테고 그만큼 사람들의 기대감도 있을 거라고 생각이 들어.',
  '하지만 그래도 열심히 평소처럼 열심히 하고 완벽주의만을 추구하지 않는다면 나는 다른 김연아 선수가 없더라도 예전처럼 각광받을 수 있을 거라고 나는 생각을 해.',
  '그런데 쇼트트랙의 경우에는 아까 네가 말했듯이 네 명이서 화합을 하는 게 중요하다고 생각을 한단 말이야.',
  '그래서 나는 그게 피겨스케이트와 쇼트트랙은 유사한 종목이지만 개인이 퍼포먼스를 한다는 점과 네 명이서 퍼포먼스한다 부분에 큰 차이점이 있다고 생각이 들어서 나는 그 부분에 대해서 협동이 잘 이루어지려면은 어떤 부분이 개선돼야 할까라고 생각을 했단 말이야.',
  '그래서 나는 선수 간의 화합을 전제로 경기를 진행하는 게 이상적인 방법이라는 생각이 들지만 앞서 말한 쇼트트랙 경기 중에서 따돌림이 발생한 경우는 평소에도 선수 간의 화합이 이루어지지 않은 부분이라서 마치 비즈니스 관계처럼 공사 구분을 철저히 하면서 함께 쇼트트랙을 하면은 감정적인 게 일어나지 않으니까 지금에 이전의 일의 일보다는 쪼끔 덜 원활하게 되지 않았을까라는 생각이 들어.',
  '그래서 너는 비즈니스 관계에 대해서 어떻게 생각해?',
    '난 일단 뭐든지 한 부분에 대해서 밀어주고 쇼트트랙이 아니더라도 여러 부분에서 나라가 지원을 많이 해준다면은 스포츠 강국으로 발생하고 나중에 외국인들도 우리나라에 대한 인식을 알 수 있지 않을까라는 생각이 들어.',
  '예전에 피겨스케이트가 잘 되었던 것처럼 지금 쇼트트랙도 밀어주면 될 거 같고 또 이전의 평창올림픽 때도 컬링으로 잘 되었으니까 여성 컬링에 대해서 잘 밀어주다 보면은 우리나라가 각 우리나라가 컬링의 강국도 될 수 있고 이번에 또 금메달을 딸 수 있지 않을까 라는 생각이 들어 넌 어떻게 생각해?',
    '긍정적인 시선으로 보고 있고 앞으로도 이렇게 스포츠 강국으로 하고 또 비빔밥이나 그런 음식적인 문화에서도 발달을 하다 보면은 사람들이 대한민국이라는 존재도 잘 알게 되고 그만큼 아 대한민국 정말 좋은 나라다 이렇게 생각을 하지 않을까라면서 여행도 많이 오지 않을까 싶어.',
  '여행도 많이 오지 않을까 싶어.'
]

In [3]:
# from konlpy.tag import Okt  
# tokenizer = Okt() ## fit on text 수정하기
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences

[[39, 89],
 [39, 89, 17],
 [39, 89, 17, 90],
 [39, 89, 17, 90, 18],
 [39, 89, 17, 90, 18, 6],
 [39, 89, 17, 90, 18, 6, 40],
 [39, 89, 17, 90, 18, 6, 40, 91],
 [39, 89, 17, 90, 18, 6, 40, 91, 92],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14, 96],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14, 96, 97],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14, 96, 97, 2],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14, 96, 97, 2, 98],
 [39, 89, 17, 90, 18, 6, 40, 91, 92, 93, 94, 95, 41, 14, 96, 97, 2, 98, 19],
 [39,
  89,
  17,
  90,
  18,
  6,
  40,
  91,
  92,
  93,
  94,
  95,
  41,
  14,
  96,
  97,
  2,
  98,
  19,
  20],
 [99, 1],
 [99, 1, 100],
 [99, 1, 100, 101],
 [99, 1, 100, 101, 42],
 [99, 1, 100, 101, 

In [4]:
tokenizer.word_index

{'그': 1,
 '잘': 2,
 '생각이': 3,
 '나는': 4,
 '대해서': 5,
 '또': 6,
 '생각을': 7,
 '들어': 8,
 '그래서': 9,
 '수': 10,
 '부분에': 11,
 '선수가': 12,
 '김연아': 13,
 '간의': 14,
 '하고': 15,
 '게': 16,
 '열심히': 17,
 '점과': 18,
 '거라고': 19,
 '생각해': 20,
 '된다는': 21,
 '때문에': 22,
 '있지': 23,
 '협동이': 24,
 '말이야': 25,
 '부분에서': 26,
 '네': 27,
 '명이서': 28,
 '큰': 29,
 '경우에는': 30,
 '선수': 31,
 '화합이': 32,
 '쇼트트랙이': 33,
 '어떻게': 34,
 '일단': 35,
 '있을': 36,
 '많이': 37,
 '않을까': 38,
 '평소에': 39,
 '다른': 40,
 '서로': 41,
 '않고': 42,
 '침착하게': 43,
 '원활하게': 44,
 '좋은': 45,
 '않았을까라는': 46,
 '쇼트트랙': 47,
 '제대로': 48,
 '부분이': 49,
 '경기': 50,
 '안타까운': 51,
 '넌': 52,
 '피겨스케이트와': 53,
 '쇼트트랙은': 54,
 '유사한': 55,
 '종목이지만': 56,
 '개인이': 57,
 '한다는': 58,
 '차이점이': 59,
 '있다고': 60,
 '평소에도': 61,
 '이루어지지': 62,
 '거': 63,
 '다시': 64,
 '사실': 65,
 '않은': 66,
 '그런': 67,
 '라는': 68,
 '지금은': 69,
 '은퇴해서': 70,
 '초점이': 71,
 '그래도': 72,
 '너는': 73,
 '한': 74,
 '그만큼': 75,
 '화합을': 76,
 '비즈니스': 77,
 '이전의': 78,
 '스포츠': 79,
 '강국으로': 80,
 '될': 81,
 '보면은': 82,
 '우리나라가': 83,
 '있고': 84,
 '이렇게': 85,
 '여행도

In [5]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words) #label을 one-hot-encoding 시킴
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [6]:
predictors.shape, len(inp_sequences)

((462, 52), 462)

In [7]:
predictors

array([[ 0,  0,  0, ...,  0,  0, 39],
       [ 0,  0,  0, ...,  0, 39, 89],
       [ 0,  0,  0, ..., 39, 89, 17],
       ...,
       [ 0,  0,  0, ...,  0, 86, 37],
       [ 0,  0,  0, ..., 86, 37, 87],
       [ 0,  0,  0, ..., 37, 87, 38]], dtype=int32)

In [14]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 40, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    
    
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 52, 40)            12240     
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 306)               30906     
                                                                 
Total params: 99,546
Trainable params: 99,546
Non-trainable params: 0
_________________________________________________________________


2023-05-26 09:29:05.998030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 09:29:05.999685: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 09:29:06.001704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:
model.fit(predictors, label, epochs=10, verbose=10)

Epoch 1/10


2023-05-26 09:29:09.592436: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 09:29:09.594473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 09:29:09.595528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_1/PartitionedCall]] [Op:__inference_train_function_7197]

In [18]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        print("1", token_list)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        
        predicted = model.predict(token_list, verbose=0).argmax()
        print(predicted)
        output_word = ""
        for word,index in tokenizer.word_index.items():
#             print(predicted.shape , predicted)
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [19]:
print (generate_text("쇼트트랙 선수", 5, model, max_sequence_len))

1 [47, 31]


2023-05-26 09:40:35.354853: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-05-26 09:40:35.354950: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1554 : UNKNOWN: Fail to find the dnn implementation.
2023-05-26 09:40:35.354991: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNKNOWN: Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
2023-05-26 09:40:35.355076: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNKNOWN: {{function_node __inference_gpu_lstm_with_fallback_7519_specialized_for_sequential_1_lstm_1_PartitionedCall_at___inference_predict_function_770

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_1/PartitionedCall]] [Op:__inference_predict_function_7708]

In [21]:
token_list = tokenizer.texts_to_sequences(['쇼트트랙 선수'])[0]
token_list

[47, 31]

In [20]:
token_list = [47]

token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
token_list.shape

(1, 52)

In [32]:
token_list = [47]
for i in range(max_sequence_len-2):
    token_list.insert(0,1)
    
token_list = np.array([token_list],dtype=np.float32)
token_list.shape

(1, 52)

In [33]:
tokenizer.word_index.items()

dict_items([('그', 1), ('잘', 2), ('생각이', 3), ('나는', 4), ('대해서', 5), ('또', 6), ('생각을', 7), ('들어', 8), ('그래서', 9), ('수', 10), ('부분에', 11), ('선수가', 12), ('김연아', 13), ('간의', 14), ('하고', 15), ('게', 16), ('열심히', 17), ('점과', 18), ('거라고', 19), ('생각해', 20), ('된다는', 21), ('때문에', 22), ('있지', 23), ('협동이', 24), ('말이야', 25), ('부분에서', 26), ('네', 27), ('명이서', 28), ('큰', 29), ('경우에는', 30), ('선수', 31), ('화합이', 32), ('쇼트트랙이', 33), ('어떻게', 34), ('일단', 35), ('있을', 36), ('많이', 37), ('않을까', 38), ('평소에', 39), ('다른', 40), ('서로', 41), ('않고', 42), ('침착하게', 43), ('원활하게', 44), ('좋은', 45), ('않았을까라는', 46), ('쇼트트랙', 47), ('제대로', 48), ('부분이', 49), ('경기', 50), ('안타까운', 51), ('넌', 52), ('피겨스케이트와', 53), ('쇼트트랙은', 54), ('유사한', 55), ('종목이지만', 56), ('개인이', 57), ('한다는', 58), ('차이점이', 59), ('있다고', 60), ('평소에도', 61), ('이루어지지', 62), ('거', 63), ('다시', 64), ('사실', 65), ('않은', 66), ('그런', 67), ('라는', 68), ('지금은', 69), ('은퇴해서', 70), ('초점이', 71), ('그래도', 72), ('너는', 73), ('한', 74), ('그만큼', 75), ('화합을', 76), ('비즈니스', 77), ('이전의', 78),

In [34]:
dic.item()

NameError: name 'dic' is not defined